# MLP for POS Tagging

In [1]:
import pyconll, keras, pickle, os, random, nltk, datetime, warnings, gc, urllib.request, zipfile
import numpy as np
import math
from scipy.sparse import hstack, vstack
import matplotlib.pyplot as plt
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import FastText
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, precision_score, classification_report, accuracy_score, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, GridSearchCV, learning_curve, cross_val_score
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, BatchNormalization, Dropout, Input, Activation
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.utils import np_utils
from numpy.random import seed
from sklearn.model_selection import train_test_split

# automatic module reloading
%load_ext autoreload
%autoreload 2

# For reproducibility
np.random.seed(3) 
random.seed(3)

2023-03-11 17:39:10.596207: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [44]:
# add parent directory to path for imports to work
import sys
sys.path.append('..')

# src imports
from src.utils import get_root_dir, untag
from src.parser import format_data, embeddings_init
from src.data_helpers import vectorize, preprocess_unlabelled_test_data

In [3]:
# functions 
def compare_with_test_set(predicted_data, correct_set):
    total = 0
    correct = 0
    for predicted_sentence, correct_sentence in zip(predicted_data, correct_set):
        for predicted_word, correct_word in zip(predicted_sentence, correct_sentence):
            total = total + 1
            if predicted_word[1] == correct_word[1]:
                #print(predicted_word[0], predicted_word[1], correct_word[1])
                correct = correct + 1
    
    accuracy = (correct / total) * 100
    return accuracy

In [4]:
ROOT_DIR = get_root_dir() 
POS_DIR = os.path.join(ROOT_DIR, 'dataset')
pos_train = os.path.join(POS_DIR, "train.txt")

In [5]:
train_sentences = format_data(pos_train)

In [6]:
print("Tagged sentences in train set: ", len(train_sentences))
print("Tagged words in train set:", len([item for sublist in train_sentences for item in sublist]))

Tagged sentences in train set:  8936
Tagged words in train set: 211727


In [ ]:
# # Dataset Split
# total_train = len(train_sentences)
# print(total_train)
# val_split = math.floor(0.2 * total_train)
# print(val_split)
# train_sentences, val_sentences = train_sentences[:(total_train-val_split)], train_sentences[(total_train-val_split):]
# print(len(train_sentences), len(val_sentences))
# print(len(train_sentences)+len(val_sentences))

In [ ]:
# print(train_sentences[0])

### Embeddings

In [7]:
# Initialize Embeddings
embeddings_path = os.path.join(get_root_dir(), 'dataset')
t_ini = datetime.datetime.now()
print('Initializing embeddings')
embeddings = embeddings_init(str(embeddings_path))
print('Initialiation completed')
t_fin = datetime.datetime.now()
print('Embeddings loaded in {} seconds'.format((t_fin - t_ini).total_seconds()))

Initializing embeddings
Initialiation completed
Embeddings loaded in 249.977289 seconds


In [8]:
w2c = dict()
for item in embeddings.key_to_index:
    w2c[item] = embeddings.key_to_index[item]

In [9]:
dim = embeddings.vectors.shape[1]
pad = np.zeros(dim)
np.random.seed(3)
oov = np.random.uniform(-0.25, 0.25, dim)

In [80]:
# Hyper parameters
BATCH_SIZE = 256
EPOCHS = 100
WINDOW_SIZE = 2

In [78]:
X_train, y_train = vectorize(embeddings, oov, train_sentences, window=WINDOW_SIZE)

Embeddings window method
Vectorizing Dataset...
Vectorizing train...
Dataset vectorized.
Train shape: (211727, 1500)


In [79]:
X_train.shape

(211727, 1500)

In [81]:
classes = sorted(list(set(y_train)))
print(classes)

['#', '$', "''", '(', ')', ',', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']


In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [82]:
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
# y_val = le.transform(y_val)
y_train = keras.utils.to_categorical(y_train)
# y_val = keras.utils.to_categorical(y_val)

print(y_train.shape)

(211727, 44)


In [ ]:
# model = Sequential()
# # In the first layer, we specify the input data shape

# model.add(Dense(128, activation='relu', input_dim=X_train.shape[1]))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.4))
# model.add(Dense(y_train.shape[1], activation='softmax'))

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

In [ ]:
# nb_epoch = 10
# batch_size = 128
# early_stopping = EarlyStopping(monitor = 'val_acc', patience = 5)
# history = model.fit(X_train, y_train,
#                     epochs=nb_epoch,
#                     batch_size=batch_size,
#                     shuffle=True,
#                     validation_data=(X_val, y_val),
#                     verbose=1,
#                     callbacks=[early_stopping])

In [ ]:
# X_full_train = vstack((X_train, X_val)).tocsr()
# y_full_train = np.append(y_train, y_val, axis=0)

In [ ]:
# X_train.shape

In [83]:
def get_mlp_model(input_dim):
    """
    Define a MLP model
    """
    model = Sequential()

    model.add(Dense(512, activation='relu', input_dim=input_dim))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [84]:
model = get_mlp_model(X_train.shape[1])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               768512    
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 64)                32832     
_________________________________________________________________
dropout_10 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 128)               8320      
_________________________________________________________________
dropout_11 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 44)               

In [85]:
model.fit(X_train, y_train,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    verbose=1,
                   ) 

2023-03-11 19:20:45.077315: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1270362000 exceeds 10% of free system memory.


Epoch 1/100
828/828 [==============================] - 5s 5ms/step - loss: 1.1264 - accuracy: 0.6968
Epoch 2/100
828/828 [==============================] - 4s 5ms/step - loss: 0.2323 - accuracy: 0.9313
Epoch 3/100
828/828 [==============================] - 4s 5ms/step - loss: 0.1649 - accuracy: 0.9509
Epoch 4/100
828/828 [==============================] - 4s 5ms/step - loss: 0.1264 - accuracy: 0.9608
Epoch 5/100
828/828 [==============================] - 4s 5ms/step - loss: 0.1060 - accuracy: 0.9674
Epoch 6/100
828/828 [==============================] - 4s 5ms/step - loss: 0.0861 - accuracy: 0.9728
Epoch 7/100
828/828 [==============================] - 4s 5ms/step - loss: 0.0787 - accuracy: 0.9753
Epoch 8/100
828/828 [==============================] - 4s 5ms/step - loss: 0.0678 - accuracy: 0.9786
Epoch 9/100
828/828 [==============================] - 4s 5ms/step - loss: 0.0573 - accuracy: 0.9819
Epoch 10/100
828/828 [==============================] - 4s 5ms/step - loss: 0.0550 - accura

In [21]:
tf.debugging.set_log_device_placement(True)

In [ ]:
# model.save("my_model.h5")

In [ ]:
# model = keras.models.load_model(save_format='h5', "my_model.h5")

In [38]:
pos_test = os.path.join(POS_DIR, "test.txt")

test_sentences = format_data(pos_test, False)

dim = embeddings.vectors.shape[1]
pad = np.zeros(dim)

In [39]:
pos_test_correct = os.path.join(POS_DIR, "test_labelled.txt") 
correct_test_sen = format_data(pos_test_correct)

In [40]:
def compare_results(predicted_sen, correct_sen, acc = True):
    total = 0
    correct_total = 0
    if acc:
        for predict, correct in zip(predicted_sen, correct_sen):
                if predict[1] == correct[1]:
                    correct_total = correct_total + 1
                total = total + 1
        acc = correct_total / total
        return acc
    else:
        for predict, correct in zip(predicted_sen, correct_sen):
                if predict[1] == correct[1]:
                    print(f"{predict[1]} \t {correct[1]}")
                else:
                    print(f"{predict[1]} \t {correct[1]} <------ Error")

In [86]:
test_sentences = format_data(pos_test, False)

preprocessed_test_data = preprocess_unlabelled_test_data(embeddings, oov, test_sentences, window=WINDOW_SIZE)

In [87]:
predicted_data = []
arg_max_dict = []
def test_set_predictions(preprocessed_test_data, test_sentences):
    for sentence in preprocessed_test_data:
        predict_x=model.predict(sentence, batch_size=1, verbose=0) 
        predict_x = np.argmax(predict_x, axis=1)
        arg_max_dict.append(predict_x)
        
    for index in range(len(test_sentences)):
        predicted_sen = list(zip(test_sentences[index], le.inverse_transform(arg_max_dict[index])))
        predicted_data.append(predicted_sen)


In [88]:
test_set_predictions(preprocessed_test_data, test_sentences)

In [ ]:
predicted_data[:2]

In [89]:
compare_with_test_set(predicted_data, correct_test_sen)

96.12892331722144

In [ ]:
def generate_labelled_data(file_name):
    f = open(file_name, "w")
    for sentence in predicted_data:
        for word, pos in sentence:
            f.write(f"{word} {pos}\n")
        f.write(f"\n")
    f.close()